# [모듈 1.1] CodeCommit 리파지토리 생성

이 노트북은 CodeCommit 리파지토리를 생성하고, codecommit 폴더를 리파지토리에 Push 하는 노트북 입니다.

## 참고 자료
- [Quick guide to start using AWS CodeCommit via AWS CLI](https://dev.to/tiamatt/quick-guide-to-start-using-aws-codecommit-via-aws-cli-1jg5)
- 개발자 가이드
    - [Setup for HTTPS users using Git credentials](https://docs.aws.amazon.com/codecommit/latest/userguide/setting-up-gc.html)
    - [Create an AWS CodeCommit repository](https://docs.aws.amazon.com/codecommit/latest/userguide/how-to-create-repository.html#how-to-create-repository-cli)
    



# 1. 환경 확인

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [2]:
%%sh 
git --version

git version 2.37.1


In [3]:
%%sh
aws --version

aws-cli/1.27.20 Python/3.8.12 Linux/5.10.102-99.473.amzn2.x86_64 botocore/1.29.32


# 2. 파라미터 로딩

In [4]:
%store -r code_pipeline_serving_config_json_path

from common_utils import load_json

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)

import json
print (json.dumps(code_pipeline_serving_dict, indent=2))

repository_name = code_pipeline_serving_dict["serving_code_repo_name"]

{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/MLOps-CodePipelineRole",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/MLOps-CodeBuildRole",
  "project_prefix": "CodePipeline-Serving-NCF",
  "region": "ap-northeast-2",
  "account_id": "057716757052",
  "serving_code_repo_name": "ncf-serving-repo",
  "code_build_project_name": "ncf-serving-sm-pipeline",
  "bucket": "sagemaker-ap-northeast-2-057716757052",
  "code_pipeline_name": "ncf-serving-code-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "endpoint_name": "ncf-codepipeline-endpoint",
  "branch_name": "master",
  "update_time": "2022-12-22-07-39-12"
}


In [5]:
source_code = 'codecommit'


# 3.리파지토리 생성

In [6]:
%%sh -s {repository_name}
repository_name=$1
aws codecommit create-repository --repository-name $repository_name \
--repository-description "My inference repository" 

{
    "repositoryMetadata": {
        "accountId": "057716757052",
        "repositoryId": "eeece452-fe71-4dfe-b34e-484e78d74d42",
        "repositoryName": "ncf-serving-repo",
        "repositoryDescription": "My inference repository",
        "lastModifiedDate": 1671694959.039,
        "creationDate": 1671694959.039,
        "cloneUrlHttp": "https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-serving-repo",
        "cloneUrlSsh": "ssh://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-serving-repo",
        "Arn": "arn:aws:codecommit:ap-northeast-2:057716757052:ncf-serving-repo"
    }
}


# 4. 리파지토리 로컬에 클로닝
### 아래를 위의 결과에서 카피해서 대체 해주세요.
#### clone_url =   "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/mlops-ncf-serving-pipeline"


In [7]:
import boto3

clone_url = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('cloneUrlHttp')
print(clone_url)


https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-serving-repo


In [8]:
%%sh -s {clone_url} {repository_name}

clone_url=$1
repository_name=$2

rm -rf $repository_name

git clone $clone_url 

Cloning into 'ncf-serving-repo'...


# 5. 소스 코드 복사

In [9]:
%%sh -s {repository_name} {source_code}
repository_name=$1
source_code=$2

cp -r $source_code/* $repository_name
ls $repository_name

codebuild-buildspec.yml
CONTRIBUTING.md
img
LICENSE
pipelines
README.md
sagemaker-pipelines-project.ipynb
setup.cfg
setup.py


# 6. 코드 파일을 리파지토리에 푸시

In [10]:
%%sh -s {repository_name} 
repository_name=$1
cd $repository_name
echo $PWD
git add .
git commit -m"Add new files"
git push origin master


/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/5_sm-serving-codepipeline/ncf-serving-repo
[master (root-commit) ff33682] Add new files
 Committer: EC2 Default User <ec2-user@ip-172-16-26-229.ap-northeast-2.compute.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 22 files changed, 1414 insertions(+)
 create mode 100644 CONTRIBUTING.md
 create mode 100644 LICENSE
 create mode 100644 README.md
 create mode 100644 codebuild-buildspec.yml
 create mode 100644 img/pipeline-full.png
 create mode 100644 pipelines/.ipynb_checkpoints/run_pipeline-checkpoint.py
 create mode 100644 pipelines/__init__.py
 create mod

remote: Validating objects: 100%        
To https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-serving-repo
 * [new branch]      master -> master


# 7. 변수 저장

In [11]:
serving_repository_name = repository_name
%store serving_repository_name

Stored 'serving_repository_name' (str)


# 8. Code Commit 생성 확인

![serving_code_commit.png](img/serving_code_commit.png)